# Two-Point Correlation Functions from Gaussian Random Fields

This notebook is designed to examine two-point correlation functions for stellar streams. It builds off of a previous discussion of Gaussian Random Fields in the context of CCD fringing [here](https://github.com/kadrlica/sandbox/blob/master/notebooks/GaussianRandomFields.ipynb). The goal (assuming we do our math correctly) is that we should be able to define a power spectrum, generate a Gaussian Random Field from that power spectrum, calculate the 2-pt correlation function on that field, and get back the original power spectrum.

**WARNING**: This is a work in progress, and it is not working yet...

In [ ]:
import numpy as np
import scipy
import pylab as plt
import scipy.ndimage as nd
from functools import partial
import scipy.interpolate
import scipy.integrate

In [ ]:
import treecorr

In [ ]:
def PowerSpectrum(k, k0=0.1, alpha=-1.5):
    """Power spectrum of the form:
    
    P(k) = (k/k0)**alpha * np.exp(-(k/k0)**2)
    
    Parameters
    ----------
    k : wave number
    k0 : scale wave number
    alpha : slope of power law at small scales
    
    Returns
    -------
    P : power spectrum
    """
    ksq = k**2
    return (ksq/k0**2)**(alpha/2.) * np.exp(-ksq/k0**2)

def x0_to_k0(x0, nx, ny):
    return np.sqrt(x0**2/(nx*ny))

def k0_to_x0(k0, nx, ny):
    return np.sqrt(k0**2 * (nx*ny))

In [ ]:
def kGaussianRandomField(Pk, nx=512, ny=512, seed=None):
    """ Gaussian Random Field in k-space drawn from a power spectrum, P(k).
    
    Parameters
    ----------
    Pk : power spectrum, P(k)
    nx : number of bins in x-space
    ny : number of bins in y-space
    seed: random seed
    
    Returns
    -------
    kx, ky, Ak : spacing in kx, ky and amplitude, Ak
    """
    size = (nx,ny)

    # Transform to k-space
    kvx,kvy = np.fft.fftfreq(nx),np.fft.fftfreq(ny)
    kx,ky = np.meshgrid(kvy, kvx, sparse=True, copy=False)

    # Choose only real values of k
    ksq = kx**2 + ky**2
    m = ksq > 0

    # Generate random numbers
    gen = np.random.RandomState(seed=seed)
    phase = 2 * np.pi * gen.uniform(size=size)
    norm = gen.normal(size=size)

    Ak = np.zeros(size,complex)
    Ak[m] = norm[m] * np.exp(1.j * phase[m]) * Pk(np.sqrt(ksq[m]))
    return kx,ky,Ak

In [ ]:
def GaussianRandomField(Pk, nx=512, ny=512):
    """Generate a Gaussian Random Field in real-space from a power spectrum, P(k)
    
    Parameters
    ----------
    Pk : Power spectrum P(k)
    nx : x-dimension in real space
    ny : y-dimension in real space
    
    Returns
    -------
    (kx,ky,Ak) : k-space components and normalization
    (xx,yy,Ax) : real-space components and normalization
    """
    xx,yy = np.meshgrid(np.arange(nx), np.arange(ny), sparse=True, copy=False)
    kx,ky,Ak = kGaussianRandomField(Pk, nx, ny)
    Ax = np.fft.ifft2(Ak)
    return (kx,ky,Ak),(xx,yy,Ax)

In [ ]:
# fftfreq doesn't sort output, so hard to plot
def draw_kspace(kx, ky, Ak, **kwargs):
    """ Plot the real component of the Gaussian Random Field in k-space.
    
    Parameters
    ----------
    kx, ky : k-space dimensions
    Ak : k-space amplitude
    kwargs : passed to matplotlib pcolormesh
    
    Returns
    -------
    None
    """
    kwargs.setdefault('vmin',-1)
    kwargs.setdefault('vmax',1)
    
    idx_kx = np.argsort(kx,axis=1)
    idx_ky = np.argsort(ky,axis=0)

    ax = plt.gca()
    im = ax.pcolormesh(kx[0,idx_kx],ky[idx_ky],Ak[idx_kx,idx_ky].real,**kwargs)
    plt.colorbar(im, ax=ax);
    ax.set_title("$k$-space (real)")
    ax.set_xlim(kx.min(),kx.max()); ax.set_ylim(ky.min(),ky.max())
    plt.xlabel('$k_x$') 
    plt.ylabel('$k_y$')

In [ ]:
def draw_xspace(xx, yy, Ax, **kwargs):
    """ Plot the real component of the Gaussian Random Field in x-space.
    
    Parameters
    ----------
    xx, yy : position-space dimensions
    Ax : position-space amplitude
    kwargs : passed to matplotlib pcolormesh
    
    Returns
    -------
    None
    """
    ax = plt.gca()
    im = ax.pcolormesh(xx,yy,Ax.real,**kwargs); plt.colorbar(im, ax=ax)
    ax.set_xlim(xx.min(),xx.max()); ax.set_ylim(yy.min(),yy.max())
    ax.set_title("$x$-space (real)")
    ax.set_xlabel('$x$'); ax.set_ylabel('$y$')

In [ ]:
def draw_power_spectrum(kx, ky, Ak):
    """ Draw the power spectrum in k-space.
    
    Parameters
    ----------
    kx, ky : k-space dimensions
    Ak : k-space amplitude

    Returns
    -------
    None
    """
    ksq = kx**2 + ky**2

    bins = np.linspace(ksq.min(),ksq.max(),100)
    centers = (bins[1:]+bins[:-1])/2.
    labels = np.digitize(ksq,bins)

    # Take the complex conjugate of the k-space matrix
    Ak_sq = np.abs(Ak)**2
    index = np.unique(labels)
    mean = nd.mean(Ak_sq.flatten(),labels=labels.flatten(),index=index)

    ax = plt.gca()
    ax.set_xscale('log'); ax.set_yscale('log')
    ax.plot(bins, mean, label='Random Generation')
    ax.set_xlabel(r'$k^2$'); 
    ax.set_ylabel(r'$ \langle |A(k)|^2 \rangle$')
    ax.set_title("Power Spectrum")
    ax.legend()

In [ ]:
# Create and draw the power spectrum
Pk = partial(PowerSpectrum, k0=0.1, alpha=-1.5)
(kx,ky,Ak),(xx,yy,Ax) = GaussianRandomField(Pk)

k = np.sqrt(np.logspace(-3,-0.3,100))

fig,ax = plt.subplots(1,3,figsize=(20,4))
plt.sca(ax[0]); draw_power_spectrum(kx, ky, Ak)
plt.loglog(k**2, Pk(k)**2, label='$P(k)^2$'); plt.legend()
plt.sca(ax[1]); draw_kspace(kx,ky,Ak)
plt.sca(ax[2]); draw_xspace(xx,yy,Ax)

### Comparing Power Spectrum to 2pt Density Correlation Function

To complete the loop, we would like to be able to compare the power spectrum to the 2D, 2pt correlation function. Ideally, we'd be able to analytically predict one from the other and then validate on the measurement.

Following some random reference from Martin White [here](http://mwhite.berkeley.edu/DES/two_point.pdf), the dimensionaless (3D) power spectrum:

$ \Delta^2(k) \equiv \frac{k^3 P(k)}{2\pi^2}$


$ \xi(r) \equiv \int \frac{dk}{k} \Delta^2(k) j_0(kr) = \int \frac{dk}{k} \frac{k^3 P(k)}{2\pi^2} \frac{sin(kr)}{kr} = \frac{1}{2\pi^2} \int dk k^2 P(k) \frac{sin(kr)}{kr}$

The power spectrum is commonly understood as the Fourier transform of the autocorrelation function, $\xi(r)$, such that:

$\xi(r) = \int \frac{d^3 k}{(2\pi)^3} P(k) e^{i {\bf k} \cdot {\bf r}} $

We will start by calculating the 2pt density correlation using the Gaussian random rield and a uniform random field. For the analysis of real data, we may want to do this directly on the points (i.e., galaxies or stars) rather than the field, so we'll explore that later.

In [ ]:
# Build the treecorr catalogs
_xx,_yy = np.meshgrid(xx.flat,yy.flat)

# Random uniform field
Ar = np.random.uniform(low=Ax.real.min(), high=Ax.real.max(), size=_xx.shape)

cat = treecorr.Catalog(x=_xx.flat,y=_yy.flat,k=Ax.real.flat)
rand = treecorr.Catalog(x=_xx.flat,y=_yy.flat,k=Ar.flat)

# Calculate the correlation functions
kwargs = dict(nbins=50, min_sep=1, max_sep=1024)
kk = treecorr.KKCorrelation(**kwargs)
kk.process(cat)
rr = treecorr.KKCorrelation(**kwargs)
rr.process(rand)
sep = np.exp(kk.meanlogr)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(18,6))

ax[0].pcolormesh(Ax.real); ax[0].set_title('Gaussian Random Field')
ax[1].pcolormesh(Ar.real); ax[1].set_title('Uniform Random Field')
ax[2].loglog(sep, kk.xi, label="Gaussian Random Field")
ax[2].loglog(sep, rr.xi, label="Uniform Random Field")
ax[2].set_title("Correlation Function")
ax[2].set_xlabel("Separation (pix)"); ax[2].set_ylabel(r"$\xi(r)$")
ax[2].legend(fontsize=10)

To compare the 2D, 2pt correlation function to the power spectrum, we will need to do some integrals....

In [ ]:
# Let's try calculating the fourier transform of the power spectrum
def integrand3d(k, r=10):
    # 1/2*pi**2 * k**2 * P(k) * sin(kr)/kr
    return (1/(2*np.pi**2)) *  k**2 * Pk(k) * np.sin(k*r)/k*r
    
def integrand2d(k, r=10):
    # Let's try a guess at the 2D fourier transform
    # 1/2*pi * k * P(k) * sin(kr)/kr
    return (1/(2*np.pi)) *  k * Pk(k) * np.sin(k*r)/k*r

#xi_r = np.array([scipy.integrate.quad(integrand2d,k.min(),k.max(),args=(_r))[0] for _r in sep])
r_range = np.logspace(np.log10(sep.min()), np.log10(sep.max()), 100)
xi_r = np.array([scipy.integrate.quad(integrand2d,k.min(),k.max(),args=(_r))[0] for _r in r_range])

In [ ]:
# Ok, so how did we do...?

# First attempt, assuming, we really have xi(r)
plt.figure()
plt.loglog(sep,kk.xi, '-')
plt.loglog(r_range,xi_r)
plt.ylabel(r'$\xi(r)$'); plt.xlabel(r'$r$')

# Ok, maybe we have xi_r * r**2?
plt.figure()
plt.loglog(sep,kk.xi, '-')
plt.loglog(r_range,xi_r/r_range**2)
plt.ylabel(r'$\xi(r)$'); plt.xlabel(r'$r$')

# Ok, maybe there is more we are missing...
plt.figure()
plt.loglog(sep,kk.xi, '-')
plt.loglog(2.3*r_range, 1/(10*15) * xi_r/(r_range)**2)
plt.ylabel(r'$\xi(r)$'); plt.xlabel(r'$r$')

In [ ]:
# Build an interpolation for the 2pt correlation function
interp = scipy.interpolate.interp1d(sep, kk.xi)
x = np.linspace(sep.min(), sep.max(),1000)
plt.figure()
plt.loglog(sep,kk.xi, 'o')
plt.loglog(x, interp(x),'-')
plt.xlabel("Separation (pix)"); plt.ylabel(r"$\xi$")
plt.title("2pt Correlation Function Interpolation")

In [ ]:
# Define the integrand
def func(r,k=0.1):
    return np.sqrt(2/np.pi) * interp(r) * r**2 * np.sin(k*r)/k*r


In [ ]:
plot_power_spectrum(xx,yy,Ax)

In [ ]:
_kx,_ky = np.meshgrid(kx.flat,ky.flat)
cat = treecorr.Catalog(x=_kx.flat,y=_ky.flat,k=Ak.real.flat)

kwargs = dict(nbins=25, min_sep=0.01, max_sep=1)
kk = treecorr.KKCorrelation(**kwargs)
kk.process(cat)

xi = kk.xi  
sep = np.exp(kk.meanlogr)

plt.figure()
plt.loglog(sep,xi)
plt.xlabel("Separation (pix)"); plt.ylabel(r"$\xi$")

### Correlation function from points

For the analysis of a small number of points (e.g., galaxies or stars), we may want to calculate the 2D, 2pt correlation function directly for the positions of those points rather than from the density field. Let's try to do that here.

In [ ]:
# Generation random points from a Poisson draw of the Gaussian Random field
mean = 1.0
lam = mean + Ax.real
counts = np.random.poisson(lam=lam, size=Ax.shape)
print(f"Simulated {counts.sum()} points.")
plt.imshow(counts)
plt.colorbar()

In [ ]:
# Generate the uniform randoms
_xx,_yy = np.meshgrid(xx.flat,yy.flat)
x = np.repeat(_xx.flatten(),counts.flatten())
y = np.repeat(_yy.flatten(),counts.flatten())
rand_x = np.random.uniform(xx.min(),xx.max(),size=len(x))
rand_y = np.random.uniform(yy.min(),yy.max(),size=len(y))

print("Array size:", len(counts.flat))
print("Number of counts:", counts.sum())
print("Number of points:", len(x),len(y))
print("Number of randoms:", len(rand_x),len(rand_y))

In [ ]:
# Create catalogs of data and randoms
cat  = treecorr.Catalog(x=x, y=y)
rand = treecorr.Catalog(x=rand_x, y=rand_y)

kwargs = dict(nbins=25, min_sep=1, max_sep=1024)
dd = treecorr.NNCorrelation(**kwargs)
dd.process(cat)
rr = treecorr.NNCorrelation(**kwargs)
rr.process(rand)
dr = treecorr.NNCorrelation(**kwargs)
dr.process(cat,rand)
xi,varxi = dd.calculateXi(rr=rr,dr=dr)

sep = np.exp(dd.meanlogr)

plt.figure()
plt.loglog(sep,xi)
plt.xlabel("Separation (pix)"); plt.ylabel(r"$\xi$")

In [ ]:
# Reminder: np.random.multinomial()